# Behind the pipeline (PyTorch)

Install the Transformers and Datasets libraries to run this notebook.

In [3]:
! pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 264 kB 7.5 MB/s 
     |████████████████████████████████| 2.6 MB 12.7 MB/s 
     |████████████████████████████████| 76 kB 5.6 MB/s 
     |████████████████████████████████| 118 kB 34.2 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 243 kB 54.3 MB/s 
     |████████████████████████████████| 3.3 MB 49.1 MB/s 
     |████████████████████████████████| 636 kB 44.9 MB/s 
     |████████████████████████████████| 895 kB 56.4 MB/s 
     |████████████████████████████████| 1.1 MB 58.9 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
# Run sentiment Classification through the pipeline API

from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier([
    "I am excited to be working through the first Huggingface course.", 
    "I am not enjoying this hot and muggy weather.",
])

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9989171028137207},
 {'label': 'NEGATIVE', 'score': 0.9992421865463257}]

In [5]:
# Reproducing pipeline API behavior

In [6]:
# Load Tokenizer

from transformers import AutoTokenizer

checkpoint = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [7]:
# Pass text sequences to tokenizer, pad & truncate, and return as Pytorch tensors

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[    0,   100,   348,    57,  2445,    13,    10, 30581,  3923, 34892,
           768,   127,  1086,   301,     4,     2],
        [    0,   100,  4157,    42,    98,   203,   328,     2,     1,     1,
             1,     1,     1,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [8]:
# Loading model

from transformers import AutoModel

checkpoint = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModel.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment were not used when initializing RobertaModel: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

In [9]:
# Pass our Pytorch tensors to the model and look at hidden state

outputs = model(**inputs)

# last hidden state 0="batch_size", 1="sequence_length", 2="hidden_size"
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


In [10]:
# Load model with sequenece classification head included

from transformers import AutoModelForSequenceClassification

checkpoint = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [11]:
# Output from model with sequence classification head will be 0="batch_size",1="num_classes"

print(outputs.logits.shape)

torch.Size([2, 3])


In [12]:
# The outputs of the model are given in logits

print(outputs.logits)

tensor([[-0.4754,  1.0974, -0.5492],
        [ 3.1936, -0.8988, -2.3628]], grad_fn=<AddmmBackward>)


In [13]:
# Use softmax to convert logits to probabilities

import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.1482, 0.7142, 0.1376],
        [0.9799, 0.0164, 0.0038]], grad_fn=<SoftmaxBackward>)


In [14]:
# Map output probabilities to labels

model.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}